# Fine tuning 

In [1]:
from datasets import load_dataset

dataset = load_dataset("mrm8488/CHISTES_spanish_jokes", split="train")
dataset

/home/kevin/.pyenv/versions/3.9.15/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['id', 'text', 'keywords', 'funny', 'category'],
    num_rows: 2419
})

In [2]:
dataset.set_format('pandas')
df = dataset.to_pandas()
df.head(10)

,id,text,keywords,funny,category
0,0,"- ¡Rápido, necesitamos sangre!\n- Yo soy 0 pos...",sangre,1,otros
1,1,- ¿Cuál es el mejor portero del mundial? \n- E...,"futbol,porteros",1,otros
2,2,El otro día unas chicas llamarón a mi puerta y...,"dinero,agua",1,otros
3,3,"- Andresito, ¿qué planeta va después de Marte?...",planetas,1,profesiones
4,4,- ¿Por qué Bob Esponja no va al gimnasio? \n- ...,"esponja,gimnasios",1,otros
5,5,Van dos ciegos y le dice uno al otro: \n- Ojal...,ciegos,1,otros
6,6,Noticia de última hora!! \n\nMuere una suegra ...,"canarias,coches,noticias",2,familia
7,7,"– Mamá, mamá, en el colegio dicen que estoy lo...","locos,sillas",1,familia
8,8,"– Mamá, mamá, ¿me haces un bocata de jamón?\n–...","madres,jamón",1,otros
9,9,- Qué pasa si te expulsan de cuatro univerdade...,"universitarios,universidades",1,otros


## Preparando el conjunto de datos

In [3]:
import torch
import transformers
import keras
import accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch.nn as nn
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from typing import Optional, Tuple
import pandas as pd
import numpy as np

2024-09-28 20:50:50.321901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-28 20:50:50.342371: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-28 20:50:50.348538: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-28 20:50:50.364823: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-28 20:50:51.555405: W tensorflow/compiler/tf2

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "DeepESP/gpt2-spanish"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model

/home/kevin/.pyenv/versions/3.9.15/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [5]:
def preprocess_function(max_len):
    def _preprocess_function(examples):
        return tokenizer(
            examples['text'],
            max_length=max_len,
            truncation=True,
            padding='longest',  # Usa el padding más corto posible para cada lote
            return_tensors='pt'
        )
    return _preprocess_function

In [6]:
dataset.reset_format()
tokenized_dataset = dataset.map(preprocess_function(max_len=256), batched=True)
tokenized_dataset = tokenized_dataset.remove_columns([col for col in tokenized_dataset.column_names if col != 'input_ids'])
tokenized_dataset = tokenized_dataset.train_test_split(train_size=0.70)
tokenized_dataset.set_format('torch')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 1693
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 726
    })
})

In [7]:
# Muestra la primera entrada del dataset tokenizado
print(tokenized_dataset['train'][0])

# lo decodeamos
tokenizer.decode(tokenized_dataset['train'][0]['input_ids'])

{'input_ids': tensor([ 2243,  1079,  1481,   281,  1028,   268,   304,  2603,   352,  1054,
          718,   324,   289,   297,  5322,   314,  1583,   304,   722, 14063,
        43176,   333,    21,  1345,   297,   299, 25840,    35,   208,    22,
          916,  2243,  4435,   649,   462, 13983,    10,   230,   208,    22,
         1170,    21,   396, 16967,   313,   289,   304,  1665,    79,  2879,
          299,  7837,   307,   304,  2603,  4461,   208,    22,   576,    66,
          420,   788, 29493,    40,   208,    22,   670,   599,  2603, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 

'Un hombre va a casa de un amigo por primera vez y en el salón se encuentra un gran león disecado, ante el que comenta:\n- ¡Un magnifico ejemplar! \n- Si, lo cacé en un safari que hice con un amigo mío\n- ¿Y como está relleno?\n- Con mi amigo<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|en

In [8]:
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

epochs = 15
batch_size = 8
logging_steps = len(tokenized_dataset['train']) // batch_size

# Definimos los parámetros globales de entrenamiento
training_args = TrainingArguments(
    output_dir='./hf-gpt',
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    learning_rate=2e-5,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    disable_tqdm=False,
    logging_steps=logging_steps
)

# Y definimos el entrenador, especificando el modelo, datasets y el tokenizador
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer
)

/home/kevin/.pyenv/versions/3.9.15/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
%%time
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.478400,3.282733
2,3.097500,3.235161
3,2.874500,3.221569
4,2.696200,3.224361
5,2.530500,3.236305
6,2.391200,3.245150
7,2.277300,3.258430
8,2.176000,3.270936
9,2.089200,3.283436
10,2.014200,3.296333


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


CPU times: user 2d 14h 32min 22s, sys: 2h 54min 30s, total: 2d 17h 26min 53s
Wall time: 2h 9min 18s


TrainOutput(global_step=3180, training_loss=2.3304054716098235, metrics={'train_runtime': 7746.6859, 'train_samples_per_second': 3.278, 'train_steps_per_second': 0.41, 'total_flos': 3317755576320000.0, 'train_loss': 2.3304054716098235, 'epoch': 15.0})

In [34]:
# Guardamos el modelo (solo correr si se vuelve a entrenar el modelo)
trainer.save_model('trained-gpt')
tokenizer.save_pretrained('trained-gpt')

('trained-gpt/tokenizer_config.json',
 'trained-gpt/special_tokens_map.json',
 'trained-gpt/vocab.json',
 'trained-gpt/merges.txt',
 'trained-gpt/added_tokens.json',
 'trained-gpt/tokenizer.json')

In [9]:
# Correr para cargar el modelo pre-entrenado
model = AutoModelForCausalLM.from_pretrained('trained-gpt')
tokenizer = AutoTokenizer.from_pretrained('trained-gpt')

In [10]:
text = 'Habia una vez un hombre que'
best = 10

In [11]:
with torch.no_grad():
    tokens = tokenizer(text, return_tensors='pt')['input_ids'].to(device)
    print("Dimensiones de la entrada:", tokens.shape)
    output = model(input_ids=tokens)
    print("Dimensiones de la salida:", output.logits.shape)
    output = output.logits[0, -1, :]
    print("Dimensiones del último token de la secuencia:", output.shape)
    probs = torch.softmax(output, dim=-1)
    print("Dimensiones de la probabilidad de los tokens:", probs.shape)
    sorted_probs = torch.argsort(probs, dim=-1, descending=True)
    print({tokenizer.decode(token): f"{prob.cpu().numpy() * 100:.2f}%" for token,
          prob in zip(sorted_probs[:best], probs[sorted_probs[:best]])})

Dimensiones de la entrada: torch.Size([1, 7])
Dimensiones de la salida: torch.Size([1, 7, 50257])
Dimensiones del último token de la secuencia: torch.Size([50257])
Dimensiones de la probabilidad de los tokens: torch.Size([50257])
{' se': '13.33%', ' estaba': '12.95%', ' le': '3.55%', ' iba': '3.28%', ' dice': '2.67%', ' en': '2.60%', ' era': '2.53%', ' tenía': '2.31%', ' pasaba': '2.17%', ' había': '2.08%'}


In [14]:
from typing import List, Tuple, Optional


def generate(
    model: nn.Module,  # El modelo de lenguaje neuronal utilizado para generar el texto
    # El tokenizador que convierte el texto en tokens para el modelo
    tokenizer: PreTrainedTokenizerBase,
    start: str,  # Texto de inicio que da el punto de partida para generar el chiste
    max_length: int = 50,  # Número máximo de tokens que se generarán
    eps: float = 0.5,  # Parámetro para la estrategia e-greedy de selección de tokens
    top_n: int = 5,  # Número de tokens con mayor probabilidad a considerar
    # Indica si se deben devolver los detalles de las iteraciones
    return_iterations: bool = False,
    # El dispositivo donde se ejecutará el modelo (por ejemplo, CPU o GPU)
    device: str = "cpu",
    # Lista de tokens que marcan el final del chiste
    stop_tokens: List[str] = ['.', '!', '?']
) -> Tuple[str, Optional[pd.DataFrame]]:

    # Inicializamos la lista 'output' con el texto inicial proporcionado
    output = [start]
    iterations = []  # Lista para almacenar los detalles de cada iteración si se solicita
    with torch.no_grad():  # Desactiva el cálculo de gradientes para ahorrar memoria, ya que no entrenamos el modelo
        # Convierte el último texto generado en tokens y los mueve al dispositivo especificado
        input_ids = tokenizer(
            output[-1], return_tensors='pt')['input_ids'].to(device)

        # Bucle principal para generar nuevos tokens hasta alcanzar la longitud máxima
        for _ in range(max_length):
            # Obtiene los logits (probabilidades sin normalizar) del modelo
            logits = model(input_ids=input_ids).logits
            # Aplica softmax para convertir los logits en probabilidades
            probs = torch.softmax(logits[0, -1, :], dim=-1)
            # Ordena los tokens por probabilidad, en orden descendente
            sorted_tokens = torch.argsort(probs, dim=-1, descending=True)

            # Estrategia e-greedy para seleccionar el siguiente token:
            # Si el valor aleatorio es menor que 'eps', se elige el token más probable,
            # de lo contrario, se selecciona uno basado en la distribución de probabilidades.
            if np.random.random_sample(1)[0] < eps:
                next_token = sorted_tokens[0].unsqueeze(
                    dim=0)  # Selecciona el token más probable
            else:
                # Selecciona un token basado en la probabilidad
                next_token = torch.multinomial(probs, 1)

            # Convierte el token seleccionado a texto
            next_word = tokenizer.decode(next_token)

            # Verifica si el token generado es un signo de puntuación que marca el final del chiste
            if any(stop in next_word for stop in stop_tokens):
                output.append(next_word)  # Añade el token final al output
                break  # Detiene la generación, ya que se alcanzó el remate del chiste

            # Si 'return_iterations' es True, almacenamos detalles de la iteración actual para análisis
            if return_iterations:
                # El texto generado hasta el momento
                iteration = {'input': ''.join(output)}
                # Obtiene los 'top_n' tokens más probables y sus respectivas probabilidades
                best_n = sorted_tokens[:top_n].cpu().tolist()
                # Guarda la información de cada token en el DataFrame
                choices = {f'Choice #{choice+1}': f'{tokenizer.decode(token)} ({prob:.4f})'
                           for choice, (token, prob) in enumerate(zip(best_n, probs[best_n].cpu().tolist()))}
                iteration.update(choices)
                iterations.append(iteration)

            # Añade la palabra generada al 'output'
            output.append(next_word)
            # Actualiza los 'input_ids' concatenando el nuevo token generado
            input_ids = torch.cat(
                [input_ids, next_token.unsqueeze(dim=0)], dim=-1)

        # Convierte la lista de tokens generados en una cadena de texto
        output_text = ''.join(output)

        # Devuelve el texto generado y, opcionalmente, las iteraciones si se solicitó
        if not return_iterations:
            return output_text, None
        else:
            # Convierte las iteraciones en un DataFrame
            df = pd.DataFrame(iterations)
            return output_text, df


# Ejemplo de uso
prompt = 'Había una vez un hombre que'
output_text, _ = generate(model, tokenizer, prompt,
                          max_length=50, eps=0.2, device=device)
print(output_text)

Había una vez un hombre que creía en el amor, muy en las profundidades de su alma, hombre apasionado que quería ofrecerte felicidad.
